# Day 6
https://adventofcode.com/2018/day/6

In [1]:
import aocd
data = aocd.get_data(year=2018, day=6)

In [2]:
from dataclasses import dataclass
import re

In [3]:
re_point = re.compile(r'(\d+), (\d+)')

In [4]:
@dataclass(frozen=True, order=True)
class Point():
    x: int
    y: int

    def distance(self, other):
        return abs(self.x-other.x) + abs(self.y-other.y)
    
    def distances(self, pointset):
        return dict((point, self.distance(point)) for point in pointset)
    
    def closest_to(self, pointset):
        distances = self.distances(pointset)
        min_dist = min(distances.values())
        closest = [point for point in pointset if distances[point] == min_dist]
        if len(closest) == 1:
            return closest[0]
    
    def total_distance_to(self, pointset):
        distances = self.distances(pointset)
        return sum(distances.values())
    
    @classmethod
    def all_from_text(cls, text):
        return set(cls(*map(int, groups)) for groups in re_point.findall(text))
    
    @classmethod
    def constraints(cls, pointset):
        x = [point.x for point in pointset]
        y = [point.y for point in pointset]
        return (min(x), min(y), max(x), max(y))    

    @classmethod
    def non_infinite_area_sizes(cls, pointset):
        nia = dict((point, set()) for point in pointset)
        infinite = set()
        
        minx, miny, maxx, maxy = cls.constraints(pointset)
        for x in range(minx, maxx+1):
            for y in range(miny, maxy+1):
                point = cls(x, y)
                closest = point.closest_to(pointset)
                if closest:
                    nia[closest].add(point)
                    if closest not in infinite and (x in (minx, maxx) or y in (miny, maxy)):
                        infinite.add(closest)
        
        return [len(nearby) for (point, nearby) in nia.items() if point not in infinite]
    
    @classmethod
    def total_distance_to_all_grid(cls, pointset):
        grid = dict()
        
        minx, miny, maxx, maxy = cls.constraints(pointset)
        for x in range(minx, maxx+1):
            for y in range(miny, maxy+1):
                point = cls(x, y)
                grid[point] = point.total_distance_to(pointset)
        
        return grid

In [5]:
points = Point.all_from_text(data)
p1 = max(Point.non_infinite_area_sizes(points))
print('Part 1: {}'.format(p1))

Part 1: 3569


In [6]:
distances = Point.total_distance_to_all_grid(points)
p2 = sum(1 for (point, total) in distances.items() if total <= 10000)
print('Part 2: {}'.format(p2))

Part 2: 48978
